<a href="https://colab.research.google.com/github/tayfununal/Normalizing-Flows/blob/main/mse_then_hamiltonian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ode solver için gerekli kütüphaneler
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

# Deep learning için gerekli kütüphaneler 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

plt.style.use('seaborn-poster')

%matplotlib inline
"""
def F(t,s):       # Diff eq F(t,s) = 0
  return [np.cos(t), -np.sin(t)]
"""
tf.random.set_seed(39)
np.random.seed(39)

initial_state = [0., 1.]                  # t0 anındaki başlangıç değerleri
t_starting_and_last = [0, 2*np.pi]        # Fonksiyonun tanımlı olacağı alanın başlangıç ve bitiş noktasını tanımlıyoruz.
t_eval = np.arange(0, 2*np.pi, 0.01)      # t_starting_and_last ile belirlediğimiz aralıkta hesaplanacak t değerlerini bu şekilde girebiliriz, hiçbirşey girilmez ise default olarak kendisi parçalıyor.

def F(t,y):       # Diff eq F(t,s) = 0
  return [y[1], -y[0]]

sol = solve_ivp(fun= F, t_span= t_starting_and_last, y0= initial_state, t_eval= t_eval, dense_output=True)


plt.figure(figsize = (8, 8))

plt.subplot(221)
plt.plot(sol.t, sol.y[0])
plt.xlabel('t')
plt.ylabel('H[0] = sin(t)')

plt.subplot(222)
plt.plot(sol.t, sol.y[0] - np.sin(sol.t))
plt.xlabel('t')
plt.ylabel('H[0] - sin(t)')

plt.subplot(223)
plt.plot(sol.t, sol.y[1])
plt.xlabel('t')
plt.ylabel('H[1] = cos(t)')

plt.subplot(224)
plt.plot(sol.t, sol.y[1] - np.cos(sol.t))
plt.xlabel('t')
plt.ylabel('H[1] - cos(t)')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (4, 4))
plt.plot(sol.y.T[:, 0], sol.y.T[:, 1])
plt.xlabel('q')
plt.ylabel('p')
plt.show()

In [ ]:
y = sol.y.T # etiketlerimiz
t = sol.t   # t değerlerimiz

t = t.reshape(([t.shape[0],1]))
shuffle = np.concatenate((y,t), axis=1)
np.random.shuffle(shuffle)

y = shuffle[:,:2]
t = shuffle[:,2]
print('y:',y, '\nt:',t)

In [ ]:
# Model
girdi = Input(shape=(1,))

hidden = Dense(16, activation= tf.keras.activations.tanh)(girdi)
hidden = Dense(16, activation= tf.keras.activations.tanh)(hidden)
hidden = Dense(16, activation= tf.keras.activations.tanh)(hidden)
hidden = Dense(16, activation= tf.keras.activations.tanh)(hidden)

hidden = Dense(2)(hidden)

model = Model(inputs=girdi, outputs=hidden)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss= 'mse')
model.fit(x=t, y= y, epochs=500, batch_size=16)

In [ ]:
def H(z):
  return tf.reduce_sum((z**2)/2,axis=1)


def custom_loss(y_true, y_pred):
  return (2/16)*(h_0 - H(y_pred))**2

# Hamiltonian for initial values [0,1]
h_0 = H(np.array([initial_state], dtype=np.float32))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss= custom_loss)
model.fit(x=t, y= y, epochs=500, batch_size=16)

In [ ]:
t_test = np.arange(0, 2*np.pi, 0.02).reshape((315,1)) #629
pred = model.predict(t_test)

hamiltonian = H(pred)
true_hamiltonian = H(sol.y.T)

plt.figure(figsize = (8, 4))

plt.subplot(121)
plt.plot(sol.y.T[:, 0], sol.y.T[:, 1])
plt.plot(pred[:, 0], pred[:, 1], c ='red')
plt.xlabel('q')
plt.ylabel('p')

plt.subplot(122)
plt.plot(t_eval, true_hamiltonian)
plt.plot(t_test, hamiltonian, c='red')
plt.xlabel('t')
plt.ylabel('H')

plt.tight_layout()
plt.show()